<a href="https://colab.research.google.com/github/bearbearyu1223/llm-fine-tuning-playground/blob/main/test_finetune_falcon_7b_sharded_freeGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Fine-tune Falcon-7b-instruct-sharded model** on a mental health conversational dataset curated by heliosbrahma can be found on Hugging Face.
Links to both the model and dataset are in the notebook.


##Installs and imports

In [ ]:
#all installs
!pip install -q -U trl accelerate git+https://github.com/huggingface/peft.git
!pip install transformers==4.34.0
!pip install -q datasets bitsandbytes einops wandb
!pip install huggingface_hub==0.18.0

#all imports
import torch
import time
from huggingface_hub import notebook_login
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training
from transformers import TrainingArguments
from trl import SFTTrainer

#ignore warnings
import warnings
warnings.filterwarnings("ignore")

##Notebook connection to Hugging face

In [ ]:
!huggingface-cli login

In [ ]:
notebook_login()

##Loading the dataset from hugging face

In [ ]:
dataset_name = "heliosbrahma/mental_health_chatbot_dataset"
data = load_dataset(dataset_name)
data

##Loading the model and Setting up bitsandbytes config

We will use sharded version of falcon-7b-instruct model



In [ ]:
model_name = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
model.config.use_cache = False

##Loading the tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

##Setting up the LoRA config

In [ ]:
model = prepare_model_for_kbit_training(model)

lora_alpha = 32 #16
lora_dropout = 0.05 #0.1
lora_rank = 32 #64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

peft_model = get_peft_model(model, peft_config)

##Load the trainer

In [ ]:
output_dir = "falcon_7b_instruct_mental_health_model_Dec_3_23"
per_device_train_batch_size = 16 #4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 100 #100 #500
warmup_ratio = 0.03
lr_scheduler_type = "cosine" #"constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    push_to_hub=True
)

##Passing arguments to the SFTT trainer

In [ ]:
max_seq_length = 256

trainer = SFTTrainer(
    model=peft_model,
    train_dataset=data['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

In [ ]:
# upcasting the layer norms in torch.bfloat16 for more stable training
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.bfloat16)

##Train the model

You can check your training time if you are doing multiple experiments

In [ ]:
start = time.time()

In [ ]:
peft_model.config.use_cache = False
trainer.train()

In [ ]:
end=time.time()

In [ ]:
time_taken=end-start
print(time_taken)

##Save the model

In [ ]:
#trainer.save() #if you want to save your model locally

##Push to hub

In [ ]:
trainer.push_to_hub()

##Inference

In [ ]:
# Loading PEFT model
PEFT_MODEL = "Srishy/falcon7binstruct_mentalhealthmodel_oct23"
config = PeftConfig.from_pretrained(PEFT_MODEL)
peft_base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

peft_model = PeftModel.from_pretrained(peft_base_model, PEFT_MODEL)

peft_tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
peft_tokenizer.pad_token = peft_tokenizer.eos_token

In [ ]:
# Generate responses from both orignal model and fine-tuned model
def get_response(question):
  prompt = f"""
  ###Instruction: You are a mental health professional, answer the following question correctly.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'

  ###Question: {question}

  ###Response:

  """

  encoding = tokenizer(prompt, return_tensors="pt").to("cuda:0")
  outputs = model.generate(input_ids=encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = tokenizer.eos_token_id, \
                                                                                                                     eos_token_id = tokenizer.eos_token_id, attention_mask = encoding.attention_mask, \
                                                                                                                     temperature=0.1, top_p=0.1, repetition_penalty=1.2, num_return_sequences=1,))
  text_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

  #print(dashline)
  print(f'Response from original falcon_7b_instruct_sharded:\n{text_output}')

  print("*******************************************************")

  peft_encoding = peft_tokenizer(prompt, return_tensors="pt").to("cuda:0")
  peft_outputs = peft_model.generate(input_ids=peft_encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = peft_tokenizer.eos_token_id, \
                                                                                                                      eos_token_id = peft_tokenizer.eos_token_id, attention_mask = peft_encoding.attention_mask, \
                                                                                                                      temperature=0.1, top_p=0.1, repetition_penalty=1.2, num_return_sequences=1,))
  peft_text_output = peft_tokenizer.decode(peft_outputs[0], skip_special_tokens=True)

  print(f'Response from fine-tuned falcon_7b_instruct_sharded:\n{peft_text_output}')


In [ ]:
get_response("Are there cures for mental health problems?")

In [ ]:
get_response("What’s the difference between psychotherapy and counselling?")

In [ ]:
get_response("What happens in a therapy session?")

In [ ]:
get_response("Are neurofeedback and biofeedback the same thing?")

In [ ]:
get_response(" What is insomnia disorder?")